In [187]:
from pathlib import Path


In [263]:
!xflash -o ./test.bin --read-all --target=XCORE-AI-EXPLORER

Site 0 has finished successfully.        


In [264]:
data = Path('./test.bin').read_bytes()

In [265]:
pos = data.find(b'MEMLDATA')
pos

2097152

In [266]:
len(data)

4194304

In [267]:
data[pos+0:pos+209]

b'MEMLDATA63000000{"cheese":"i11000000","pi":"fda0f4940","weights":["fcdccf642","fcdcccc3d","f33338bc1","f00000000"]}\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'

In [268]:
def hexStrToByte(hexbytes):
    return (int(hexbytes[0],16) * 16) + int(hexbytes[1], 16)
def getBytesFromHex(hexbytes):
    b0 = hexStrToByte(hexbytes[0:2])
    b1 = hexStrToByte(hexbytes[2:4])
    b2 = hexStrToByte(hexbytes[4:6])
    b3 = hexStrToByte(hexbytes[6:8])
    return [b0,b1,b2,b3]
    
def hexToSizeT(hexbytes):
    bytes = getBytesFromHex(hexbytes)
    print(bytes)
    return int.from_bytes(bytes, "little", signed=False)
    


In [269]:
lenbytes = data[pos+8:pos+16].decode('ASCII')
datalen = hexToSizeT(lenbytes)
datalen

[99, 0, 0, 0]


99

In [270]:
import json

In [271]:
js = data[pos+16:pos+16+datalen]

In [272]:
js

b'{"cheese":"i11000000","pi":"fda0f4940","weights":["fcdccf642","fcdcccc3d","f33338bc1","f00000000"]}'

In [273]:
datastring = js.decode('ASCII')

In [274]:
datastring

'{"cheese":"i11000000","pi":"fda0f4940","weights":["fcdccf642","fcdcccc3d","f33338bc1","f00000000"]}'

In [298]:
obj=json.loads(datastring)

In [299]:
obj

{'cheese': 'i11000000',
 'pi': 'fda0f4940',
 'weights': ['fcdccf642', 'fcdcccc3d', 'f33338bc1', 'f00000000']}

In [200]:
len(obj['weights'])

4

In [293]:
def decodeNumber(num):
    v=0;
    if num[0] == 'i':
        numbytes = getBytesFromHex(num[1:])
        print(numbytes)
        v= int.from_bytes(numbytes, "little", signed=True)        
    elif num[0] == 'f':
        numbytes = getBytesFromHex(num[1:])
        print(numbytes)
        v= struct.unpack('f', bytes(numbytes))[0]
    return v;

decodeNumber(obj['pi'])

[218, 15, 73, 64]


3.141592502593994

In [294]:
def decodeList(lst):
    return [decodeNumber(x) for x in lst]

decodeList(obj['weights'])

[205, 204, 246, 66]
[205, 204, 204, 61]
[51, 51, 139, 193]
[0, 0, 0, 0]


[123.4000015258789, 0.10000000149011612, -17.399999618530273, 0.0]

In [297]:
def decodeObject(obj):
    for k in obj.keys():
        print(obj[k])
        if isinstance(obj[k], list):
            obj[k] = decodeList(obj[k])
        else:
            obj[k] =decodeNumber(obj[k])


In [300]:
decodeObject(obj)

i11000000
[17, 0, 0, 0]
fda0f4940
[218, 15, 73, 64]
['fcdccf642', 'fcdcccc3d', 'f33338bc1', 'f00000000']
[205, 204, 246, 66]
[205, 204, 204, 61]
[51, 51, 139, 193]
[0, 0, 0, 0]


{'cheese': 17,
 'pi': 3.141592502593994,
 'weights': [123.4000015258789, 0.10000000149011612, -17.399999618530273, 0.0]}

In [302]:
obj

{'cheese': 17,
 'pi': 3.141592502593994,
 'weights': [123.4000015258789, 0.10000000149011612, -17.399999618530273, 0.0]}